In [ ]:

# Made with <3
# by Ivan Zepeda
# github@ijzepeda-LC

In [1]:
import pandas as pd
import os

# Define path for the combined file
combined_file_path = 'C:\\Users\\ijzep\\_AI_\\CAPSTONE\\GIT_project\\Recipix\\backend\\computer_vision\\Step4_telegram\\nlp\\combined_recipes.pkl'

# Check if the combined file already exists
if os.path.exists(combined_file_path):
    # Load the combined DataFrame from the saved file
    recipes = pd.read_pickle(combined_file_path)
else:
    # Your existing code to load and combine data from individual files
    allrecipes_raw = pd.read_json('C:\\Users\\ijzep\\_AI_\\CAPSTONE\\GIT_project\\Recipix\\backend\\computer_vision\\Step4_telegram\\nlp\\recipes_raw_nosource_ar.json')
    epicurious_raw = pd.read_json('C:\\Users\\ijzep\\_AI_\\CAPSTONE\\GIT_project\\Recipix\\backend\\computer_vision\\Step4_telegram\\nlp\\recipes_raw_nosource_epi.json')
    foodnetwork_raw = pd.read_json('C:\\Users\\ijzep\\_AI_\\CAPSTONE\\GIT_project\\Recipix\\backend\\computer_vision\\Step4_telegram\\nlp\\recipes_raw_nosource_fn.json')

    allrecipes = allrecipes_raw.copy().T.reset_index().drop(columns=['index'])
    epicurious = epicurious_raw.copy().T.reset_index().drop(columns=['index'])
    foodnetwork = foodnetwork_raw.copy().T.reset_index().drop(columns=['index'])

    # Combine the data
    recipes = pd.concat([allrecipes, epicurious, foodnetwork]).reset_index(drop=True)

    # Save the combined DataFrame to a file for future use
    recipes.to_pickle(combined_file_path)

# Proceed with your analysis using the 'recipes' DataFrame


In [5]:

import string
from functools import reduce
from operator import add
# Load and preprocess your data here
# Example: loading and combining datasets
recipes = pd.read_pickle('../nlp/combined_recipes.pkl')

# Perform any cleaning or preprocessing required

print("Processing NLP on recipes")

# Cleaning
null_recs = recipes.copy().drop(columns='picture_link').T.isna().any()
rows_to_drop = recipes[null_recs].index
recipes = recipes.drop(index=rows_to_drop).reset_index(drop=True)

nc_ingred_index = [index for i, index in zip(recipes['ingredients'], recipes.index) if
                   all(j.isdigit() or j in string.punctuation for j in i)]
nc_title_index = [index for i, index in zip(recipes['title'], recipes.index) if
                  all(j.isdigit() or j in string.punctuation for j in i)]
nc_instr_index = [index for i, index in zip(recipes['instructions'], recipes.index) if
                  all(j.isdigit() or j in string.punctuation for j in i)]

index_list = [nc_ingred_index, nc_title_index, nc_instr_index]

inds_to_drop = set(reduce(add, index_list))
recipes = recipes.drop(index=inds_to_drop).reset_index(drop=True)


empty_instr_ind = [index for i, index in zip(recipes['instructions'], recipes.index) if len(i) < 20]
recipes = recipes.drop(index=empty_instr_ind).reset_index(drop=True)



# Save the preprocessed data
recipes.to_pickle('../nlp/preprocessed_recipes.pkl')
# You could also save other artifacts like tokenized text, if necessary


Processing NLP on recipes


In [5]:
from fpdf import FPDF
import os
import datetime

t = datetime.datetime.today()
D = t.strftime('%B %d, %y')
D_file = t.strftime('%d-%m-%y_%H%M%S')
pdf_folder = "./pdfs"
logo_path = "./resources/recipix_logo.png" #es desde donde se ejecuta, no del origen de la libreria

# if folder doesn't exist create it
if not os.path.exists(pdf_folder):
    os.makedirs(pdf_folder)


def is_not_valid_data(dictionary):
    for value in dictionary.values():
        if value is not None and value:
            return False
    return True


 

def save_pdf_style1(user, ingredients,recipe, dallae_img_path, other_recipes, logo_path=logo_path):
   
    pdf = FPDF()
    pdf.add_page()
    pdf.set_fill_color(245, 245, 220) #(255, 253, 208)  # RGB for Cream
    pdf.rect(0, 0, pdf.w, pdf.h, 'F')
    border_width = 1  # Width of the border
    pdf.set_line_width(border_width)
    pdf.rect(5, 5, pdf.w - 10, pdf.h - 10)  # Adjust the dimensions as needed

    # pdf=add_page_with_background('beige')  # You can change 'beige' to 'cream' or any other color
    # pdf=rounded_rect(10, 10, pdf.w - 20, pdf.h - 20, 5, 'D')  # Draw a border with rounded corners

    # pdf.set_margins(left= 15,  right=-15, top=10)
    font = "Arial"
    page_width = pdf.w
    
    SPACE=0
    # pdf.set_margins(left= (5+SPACE),  right=-(5+SPACE), top=SPACE)
    
    logo_width = 16-SPACE
    dalle_with = 30-SPACE
    
    x_coordinate = page_width - logo_width - 5

    pdf.set_font(font, "B", size=40)
    pdf.cell(0, 10, txt="Recipix", ln=True, align='C')
    pdf.set_font(font, "B", size=10)
    pdf.cell(0, 10, txt="your food coach", ln=False, align='C')
    pdf.ln(5)
    pdf.image(logo_path, w=30, x=page_width/2-logo_width)
    # pdf.image(logo2_path, x=x_coordinate, y=4, w=25)
    pdf.ln()

    pdf.set_font(font, "B", size=15)
    pdf.cell(0, 10, txt=f"Recipe for {user['first_name']}", align='C')  # Title
    pdf.ln()
    pdf.set_font(font, "BU", size=25)
    pdf.cell(0, 10, txt=recipe['title'], align='C')  # Date
    pdf.ln(15)

    pdf.image(dallae_img_path, w=60, x=page_width/2-dalle_with)
    pdf.ln()

    pdf.set_font(font, "I", size=12)
    pdf.cell(40, 10, txt="Ingredients found:", ln=False)
    pdf.set_font(font, "IU", size=12)
    pdf.cell(40, 10, txt=f"{ingredients}", ln=True)


    pdf.set_font(font, "B", size=15)
    pdf.multi_cell(0, 10, txt="Ingredients:\n")
    pdf.set_font(font, size=12)
    pdf.multi_cell(0, 10, txt=recipe['ingredients'].capitalize())
    pdf.ln()

    pdf.set_font(font, "B", size=15)
    pdf.multi_cell(0, 10, txt="Instructions:\n")
    pdf.set_font(font, size=12)
    pdf.multi_cell(0, 10, txt=recipe['instructions'].capitalize())
    pdf.ln()

    pdf.set_font(font, "B", size=15)
    pdf.multi_cell(0, 10, txt="Other Recipes:\n")
    pdf.set_font(font, size=12)
    pdf.multi_cell(0, 10, txt=other_recipes.capitalize())
    pdf.ln()

    # if path doesn't exists create it
    if not os.path.exists(pdf_folder):
        os.makedirs(pdf_folder)
    pdf_path = os.path.join(pdf_folder, f"Recipe-{recipe['title']}-for-{user['first_name']}_style1.pdf")
    pdf.output(pdf_path, 'F')  # .encode('latin-1','ignore').decode('latin-1')
    print("PDF SAVED AT", pdf_path)
    return pdf_path




def save_pdf_style2(user, ingredients,recipe, dallae_img_path, other_recipes, logo_path=logo_path):
   
    pdf = FPDF()
    pdf.add_page()
    pdf.set_fill_color(245, 245, 220) #(255, 253, 208)  # RGB for Cream
    pdf.rect(0, 0, pdf.w, pdf.h, 'F')
    border_width = 1  # Width of the border
    pdf.set_line_width(border_width)
    pdf.rect(5, 5, pdf.w - 10, pdf.h - 10)  # Adjust the dimensions as needed

    # pdf=add_page_with_background('beige')  # You can change 'beige' to 'cream' or any other color
    # pdf=rounded_rect(10, 10, pdf.w - 20, pdf.h - 20, 5, 'D')  # Draw a border with rounded corners

    # pdf.set_margins(left= 15,  right=-15, top=10)
    font = "Arial"
    page_width = pdf.w
    
    SPACE=0
    # pdf.set_margins(left= (5+SPACE),  right=-(5+SPACE), top=SPACE)
    
    logo_width = 16-SPACE
    dalle_with = 30-SPACE
    
    x_coordinate = page_width - logo_width - 5

    pdf.set_font(font, "B", size=40)
    pdf.cell(0, 10, txt="Recipix", ln=True, align='C')
    pdf.set_font(font, "B", size=10)
    pdf.cell(0, 10, txt="your food coach", ln=False, align='C')
    pdf.ln(5)
    pdf.image(logo_path, w=30, x=page_width/2-logo_width)
    # pdf.image(logo2_path, x=x_coordinate, y=4, w=25)
    pdf.ln()

    pdf.set_font(font, "B", size=15)
    pdf.cell(0, 10, txt=f"Recipe for {user['first_name']}", align='C')  # Title
    pdf.ln()
    pdf.set_font(font, "BU", size=25)
    pdf.cell(0, 10, txt=recipe['title'], align='C')  # Date
    pdf.ln(15)

    pdf.image(dallae_img_path, w=60, x=page_width/2-dalle_with)
    pdf.ln()

    pdf.set_font(font, "I", size=12)
    pdf.cell(40, 10, txt="Ingredients found:", ln=False)
    pdf.set_font(font, "IU", size=12)
    pdf.cell(40, 10, txt=f"{ingredients}", ln=True)


    pdf.set_font(font, "B", size=15)
    pdf.multi_cell(0, 10, txt="Ingredients:\n")
    pdf.set_font(font, size=12)
    pdf.multi_cell(0, 10, txt=recipe['ingredients'].capitalize())
    pdf.ln()

    pdf.set_font(font, "B", size=15)
    pdf.multi_cell(0, 10, txt="Instructions:\n")
    pdf.set_font(font, size=12)
    pdf.multi_cell(0, 10, txt=recipe['instructions'].capitalize())
    pdf.ln()

    pdf.set_font(font, "B", size=15)
    pdf.multi_cell(0, 10, txt="Other Recipes:\n")
    pdf.set_font(font, size=12)
    pdf.multi_cell(0, 10, txt=other_recipes.capitalize())
    pdf.ln()

    # if path doesn't exists create it
    if not os.path.exists(pdf_folder):
        os.makedirs(pdf_folder)
    pdf_path = os.path.join(pdf_folder, f"Recipe-{recipe['title']}-for-{user['first_name']}_style2.pdf")
    pdf.output(pdf_path, 'F')  # .encode('latin-1','ignore').decode('latin-1')
    print("PDF SAVED AT", pdf_path)
    return pdf_path
















In [3]:


user={}
user['first_name']='Ivanz'

ingredients=['Salmon', 'orange', 'oil', 'rice', 'kimchi']
first_recipe={
    'title': 
"Mom's Kimchi Egg",
    'ingredients': 
'2 tablespoons vegetable oil; 1 cup kimchi, or to taste; 2 large eggs, beaten',

'instructions': 
'Heat oil in a skillet or wok over medium heat. Cook kimchi in hot oil until softened, about 2 minutes; add eggs. Cook and stir eggs and kimchi together until the eggs are set, 2 to 3 minutes.'
}
dallae_img_path="C:\\Users\\ijzep\\_AI_\\CAPSTONE\\GIT_project\\Recipix\\backend\\computer_vision\\Step4_telegram\\user_data\\suggested_recipe.png"

other_recipes_summary="Recipe 2: Caramelized Kimchi Relish,\n Recipe 3: Kimchi Spread"
LONG_TEXT=("lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum \n"
           "lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum\n"
           "lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum\n")
       

In [112]:
pdf_path = save_pdf_style1(user, ingredients, first_recipe, dallae_img_path, other_recipes_summary, logo_path="../resources/recipix_logo.png")


In [163]:

def split_long_text(pdf, long_text, page_width=210, cell_height=10, font_size=24, separators=[]):
    for separator in separators:
        long_text=long_text.replace(separator,'\n* ') 
    lines = long_text.split("\n")
    cell_height= 5 if font_size<=10 else cell_height #TODO: modified, not here
    split_lines = []
    for line in lines:
        while len(line) > 0:
            max_chars_per_line = 15+abs(70-(font_size*3)) #17
            # If the line is short enough, add it as is
            if len(line) <= max_chars_per_line:
                split_lines.append(line)
                break
            # If the line is too long, split at the nearest space before the limit
            split_point = line.rfind(' ', 0, max_chars_per_line)
            if split_point == -1:  # No space found, force split at max_chars_per_line
                split_point = max_chars_per_line
            # Add the split line to the list
            split_lines.append(line[:split_point].strip())
            # Remove the processed part from the line
            line = line[split_point:].lstrip()

    # Add each chunk into the PDF as a new cell
    for split_line in split_lines:
        pdf.set_x(page_width / 2)
        pdf.cell(0, cell_height, str(split_line).strip(), 0, 1, 'L')
        
        

def save_pdf_style2(user, ingredients, recipe, dallae_img_path, other_recipes):
    LOGO_TRANSPARENT_PATH='../resources/recipix_logo_transparent.png'
    LOGO_PATH='../resources/recipix_logo.png'
    
    
    # Create instance of FPDF class
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    
    # Add a page
    pdf.add_page()
    
    page_width = pdf.w
    page_height = pdf.h
    
    pdf.set_fill_color(245, 245, 220) #(255, 253, 208)  # RGB for Cream
    pdf.rect(0, 0, pdf.w, pdf.h, 'F')
    border_width = 1  # Width of the border
    pdf.set_line_width(border_width)
    pdf.rect(5, 5, pdf.w - 10, pdf.h - 10)  # Adjust the dimensions as needed
    
    
    color_purple=(152, 4, 176) #255.0.0
    color_red=(220, 50, 50)
    color_blue=(41, 26, 99)
    color_beige=(255, 226, 182)
    color_white=(255, 255, 255)
    color_black=(0,0,0)
    
    accent_color=color_purple
    second_color=color_beige
    title_color=color_blue
    banner_text_color=color_beige
    banner_color=color_red
    
    
    #HEADER-----------------------------
    # Set title
    pdf.set_y(11)
    pdf.set_font("Arial", 'B', 30)
    pdf.set_text_color(title_color[0],title_color[1],title_color[2])  
    pdf.cell(200, 10, "Recipix", 0, 1, 'C')
    pdf.set_font("Arial", 'B', 15)
    pdf.cell(200, 10, "Your food coach", 0, 1, 'C')
    
    # Draw a colored line
    pdf.set_draw_color(accent_color[0],accent_color[1],accent_color[2])  
    pdf.set_line_width(1)
    pdf.line(10, 35, 200, 35)
    
    # Add a logo
    pdf.image(LOGO_PATH, x=10, y=3, w=35)
    
    pdf.image(LOGO_TRANSPARENT_PATH, x=10, y=80, w=190)
    #------------------------------------------------------
    #INTENTO DE FOTOYTITULO
    # Set title
    pdf.set_y(40)
    pdf.set_font("Arial", 'B', 24)
    pdf.set_text_color(title_color[0],title_color[1],title_color[2])  
    pdf.set_x(page_width/2)
    split_long_text(pdf, recipe['title'], page_width=page_width, font_size=24) #TODO <<<<<<<<<<<<<<<<<<<
    
    pdf.set_font("Arial", 'B', 16)
    pdf.set_x(page_width/2)
    pdf.cell(50, 10, f"Tailored for {user['first_name']}", 0,1, 'L')
    pdf.set_x(page_width/2)
    # pdf.ln(2)
    # pdf.set_font("Arial",'', 12)
    # ingredienteses=f"Found Ingredients:\n{ingredients}"
    # split_long_text(pdf, ingredienteses, page_width=page_width , font_size=12, separators=[',',';']) #TODO <<<<<<<<<<<<<<<<<<< depende del size del font
     
     # DALL-e image food image
    pdf.image(dallae_img_path, x=15, y=40, w=page_width*0.4)
    pdf.set_draw_color(accent_color[0],accent_color[1],accent_color[2])
    pdf.rect(15, 40, page_width*0.4, page_width*0.4 )
    
    #--------------------CONTENT-----------------------
    
    # Section title: Ingredients
    # pdf.ln(45)  
    pdf.ln(2)  
    pdf.set_x(page_width/2)
    
    pdf.set_font("Arial", 'B', 20)
    pdf.set_fill_color(banner_color[0],banner_color[1],banner_color[2]) # Red fill color
    pdf.set_text_color(banner_text_color[0],banner_text_color[1],banner_text_color[2])  
    pdf.cell(70, 10, " INGREDIENTS", 0, 1, 'L', 1)
    pdf.ln(2)  
    
    # Ingredients list
    pdf.set_font("Arial", 'B', 10)
    pdf.set_text_color(0, 0, 0) # Black text color
    # pdf.multi_cell(0, 10, recipe['ingredients'])
    split_long_text(pdf, recipe['ingredients'], page_width=page_width , font_size=10, separators=[',',';']) #TODO <depende el formato que venga
     
    pdf.set_y((page_height/2)-15)
    # Section title: Instructions
    pdf.set_font("Arial", 'B', 20)
    pdf.set_fill_color(banner_color[0],banner_color[1],banner_color[2]) # Red fill color
    pdf.set_text_color(banner_text_color[0],banner_text_color[1],banner_text_color[2])  
    pdf.cell(0, 10, " INSTRUCTIONS", 0, 1, 'L', 1)
    
    # Instructions list
    pdf.set_font("Arial", 'B', 14)
    pdf.set_text_color(0,0,0)  
    pdf.multi_cell(0, 10, recipe['instructions'])
    
    pdf.ln(2)  
    
    # Section title: Other Recipes
    pdf.set_font("Arial", 'B', 18)
    pdf.set_fill_color(banner_color[0],banner_color[1],banner_color[2]) # Red fill color
    pdf.set_text_color(banner_text_color[0],banner_text_color[1],banner_text_color[2])  
    pdf.cell(0, 10, " OTHER RECIPES", 0, 1, 'L', 1)
    
    # Other recipes list
    pdf.set_font("Arial", 'B', 14)
    pdf.set_text_color(0, 0, 0) # Black text color
    pdf.multi_cell(0, 10, other_recipes)
    
    pdf.ln(2)  
    pdf.set_font("Arial", 'UI', 10)
    pdf.cell(0, 10, f"Recipe based on the ingredients:{ingredients}")
    
    # Output the PDF
    if not os.path.exists(pdf_folder):
            os.makedirs(pdf_folder)
    pdf_path = os.path.join(pdf_folder, f"Recipe-{recipe['title']}-for-{user['first_name']}_style2.pdf")
    pdf.output(pdf_path, 'F')  # .encode('latin-1','ignore').decode('latin-1')
    print("PDF SAVED AT", pdf_path)
    return pdf_path


pdf_path = save_pdf_style2(user, ingredients, first_recipe, dallae_img_path, other_recipes_summary)
       

PDF SAVED AT ./pdfs\Recipe-Mom's Kimchi Egg-for-Ivanz_style2.pdf


In [10]:
import random

def split_long_text(pdf, long_text, page_width=210, cell_height=10, font_size=24, separators=[] ):
    if(page_width>100):
        starting_point = page_width
    else:
        page_width=150
        starting_point =20
    if(type(long_text) == list):
        long_text = ", ".join(long_text)

    for separator in separators:
        long_text = long_text.replace(separator, '\n* ') #// mas de 8 items excede

    lines = long_text.split("\n")
    # cell_height = 5 if font_size <= 10 else cell_height  # TODO: modified, not here
    split_lines = []
    for line in lines:
        while len(line) > 0:
            max_chars_per_line = 15 + abs(70 - (font_size * 3))  # 17
            max_chars_per_line =  abs(page_width - (font_size * 3))  # 17
            # If the line is short enough, add it as is
            if len(line) <= max_chars_per_line:
                split_lines.append(line)
                break
            # If the line is too long, split at the nearest space before the limit
            split_point = line.rfind(' ', 0, max_chars_per_line)
            if split_point == -1:  # No space found, force split at max_chars_per_line
                split_point = max_chars_per_line
            # Add the split line to the list
            split_lines.append(line[:split_point].strip())
            # Remove the processed part from the line
            line = line[split_point:].lstrip()

    # Add each chunk into the PDF as a new cell
    for split_line in split_lines:
        pdf.set_x(starting_point / 2)
        pdf.cell(0, cell_height, str(split_line).strip(), 0, 1, 'L')
    return len(split_lines)

def save_pdf_style3(user, ingredients, recipe, dallae_img_path, other_recipes):
    LOGO_TRANSPARENT_PATH='../resources/recipix_logo_transparent.png'
    LOGO_PATH='../resources/recipix_logo.png'
    PDF_FOLDER='../user_data/pdfs'
    # Create instance of FPDF class
    pdf = FPDF(orientation='P', unit='mm', format='A4')

    # Add a page
    pdf.add_page()

    page_width = pdf.w
    page_height = pdf.h

    #------ colors -------------------
    color_purple = (152, 4, 176)  # 255.0.0
    color_red = (220, 50, 50)
    color_blue = (41, 26, 99)
    color_beige = (255, 226, 182)
    color_white = (255, 255, 255)
    color_black = (0, 0, 0)
    color_cream=(245,245,220)
    color_grey = (217, 217, 217)

    accent_color = color_purple
    second_color = color_beige
    title_color = color_blue
    banner_text_color = color_beige
    banner_color = color_red
    background_color=color_beige

    #--- PAge setup----------------------
    pdf.set_fill_color(background_color[0], background_color[1], background_color[2])  # (255, 253, 208)  # RGB for Cream
    pdf.rect(0, 0, pdf.w, pdf.h, 'F')
    border_width = 1  # Width of the border
    pdf.set_line_width(border_width)
    pdf.set_draw_color(accent_color[0], accent_color[1], accent_color[2])
    pdf.rect(5, 5, pdf.w - 10, pdf.h - 10)  # Adjust the dimensions as needed

    # HEADER-----------------------------
    # Set title
    pdf.set_y(11)
    pdf.set_font("Arial", 'B', 30)
    pdf.set_text_color(title_color[0], title_color[1], title_color[2])
    pdf.cell(200, 10, "Recipix", 0, 1, 'C')
    pdf.set_font("Arial", 'B', 15)
    pdf.cell(200, 10, "Your food coach", 0, 1, 'C')

    # Draw a colored line
    pdf.set_draw_color(accent_color[0], accent_color[1], accent_color[2])
    pdf.set_line_width(1)
    pdf.line(10, 35, 200, 35)

    # Add a logo
    pdf.image(LOGO_PATH, x=10, y=3, w=35)

    pdf.image(LOGO_TRANSPARENT_PATH, x=10, y=80, w=190)
    # ------------------------------------------------------
    # INTENTO DE FOTOYTITULO
    # Set title
    pdf.set_y(40)
    pdf.set_font("Arial", 'B', 24)
    pdf.set_text_color(title_color[0], title_color[1], title_color[2])
    pdf.set_x(page_width / 2)
    split_long_text(pdf, recipe['title'], page_width=page_width, font_size=24,cell_height=10)  # TODO <<<<<<<<<<<<<<<<<<<

    pdf.set_font("Arial", 'B', 16)
    pdf.set_x(page_width / 2)
    pdf.cell(50, 10, f"Tailored for {user['first_name']}", 0, 1, 'L')
    pdf.set_x(page_width / 2)
    # pdf.ln(2)
    # pdf.set_font("Arial",'', 12)
    # ingredienteses=f"Found Ingredients:\n{ingredients}"
    # split_long_text(pdf, ingredienteses, page_width=page_width , font_size=12, separators=[',',';']) #TODO <<<<<<<<<<<<<<<<<<< depende del size del font

    # DALL-e image food image
    pdf.image(dallae_img_path, x=15, y=40, w=page_width * 0.4)
    pdf.set_draw_color(accent_color[0], accent_color[1], accent_color[2])
    pdf.rect(15, 40, page_width * 0.4, page_width * 0.4)

    # --------------------CONTENT-----------------------

    # Section title: Ingredients
    # pdf.ln(45)
    pdf.ln(2)
    pdf.set_x(page_width / 2)

    pdf.set_font("Arial", 'B', 20)
    pdf.set_fill_color(banner_color[0], banner_color[1], banner_color[2])  # Red fill color
    pdf.set_text_color(banner_text_color[0], banner_text_color[1], banner_text_color[2])
    pdf.cell(70, 10, " INGREDIENTS", 0, 1, 'L', 1)
    pdf.ln(3)

    # Ingredients list
    pdf.set_font("Arial", 'B', 10)
    pdf.set_text_color(0, 0, 0)
    # pdf.multi_cell(0, 10, recipe['ingredients'])
    # TODO <depende el formato que venga comma is reserved for steps
    count_ings= split_long_text(pdf, recipe['ingredients'] , page_width=page_width,cell_height=5, font_size=10, separators=[ ';'])
    #--------------------------------------------------

    pdf.set_y((page_height / 2) - 15 + count_ings)
    # Section title: Instructions
    pdf.set_font("Arial", 'B', 20)
    pdf.set_fill_color(banner_color[0], banner_color[1], banner_color[2])
    pdf.set_text_color(banner_text_color[0], banner_text_color[1], banner_text_color[2])
    pdf.cell(0, 10, " INSTRUCTIONS", 0, 1, 'L', 1)

    # Instructions list
    pdf.set_font("Arial", 'B', 12)
    pdf.set_text_color(0, 0, 0)
    pdf.multi_cell(0, 6, recipe['instructions'] )
    #--------------------------------------------------
    pdf.ln(2)

    # Section title: Other Recipes
    pdf.set_font("Arial", 'B', 18)
    pdf.set_fill_color(banner_color[0], banner_color[1], banner_color[2])
    pdf.set_text_color(banner_text_color[0], banner_text_color[1], banner_text_color[2])
    pdf.cell(0, 10, " OTHER RECIPES:", 0, 1, 'L', 1)

    # Other recipes list
    pdf.set_font("Arial", 'B', 10)
    pdf.set_text_color(0, 0, 0)
    pdf.multi_cell(0, 5, other_recipes)
    #--------------------------------------------------

    pdf.ln(2)
    pdf.set_font("Arial", 'UI', 10)
    pdf.cell(0, 10, f"Recipe based on the ingredients:")
    pdf.ln()
    split_long_text(pdf, ingredients, cell_height=3, page_width=20, font_size=10 ) #se puede pasar a una segunda pagina

    # Output the PDF
    if not os.path.exists(PDF_FOLDER):
        os.makedirs(PDF_FOLDER)

    randvar = random.randint(50000, 100000)
    pdf_path = os.path.join(PDF_FOLDER, f"Recipe3-{recipe['title']}-for-{user['first_name']}_{randvar}.pdf")
    pdf.output(pdf_path, 'F')  # .encode('latin-1','ignore').decode('latin-1')
    print("PDF SAVED AT", pdf_path)
    return pdf_path
    

pdf_path = save_pdf_style3(user, ingredients, first_recipe, dallae_img_path, other_recipes_summary)
       

PDF SAVED AT ../user_data/pdfs\Recipe3-Mom's Kimchi Egg-for-Ivanz_64517.pdf


In [ ]:
import subprocess

try:
    result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True, check=True)
    print(result.stdout)
except Exception as e:
    print(f"Error: {e}")
